# SELECT
다양한 조건을 사용하여 SELECT문을 사용해본다

In [1]:
import pymysql
import pandas as pd
import numpy as np
from config import *
import uuid
import datetime 
from itertools import product

In [2]:
def query(sql, data):
    """
    DB에 SQL문 실행(query: 데이터베이스에 정보 요청)
       Args:
           sql(str): sql 구문
           data(list or tuple): sql 파라미터. ex) [val1, val2 ...]
       Returns:
           query 결과. 결과가 없으면 빈 tuple 리턴
    """
    # connect(): DB와 연결
    db = pymysql.Connect(host=MYSQL_HOST, 
                     user=MYSQL_USER, 
                     password=MYSQL_PASSWORD, 
                     database=MYSQL_DATABASE,
                     autocommit=MYSQL_AUTOCOMMIT, # Query 실행 후 자동 commit(실제 insert, update 반영)
                     charset=MYSQL_CHARSET, # 인코딩 정보
                     cursorclass=pymysql.cursors.DictCursor # DB를 조회한 결과를 Column 명이 Key인 Dictionary로 저장
                    )
    cursor = db.cursor()

    cursor.execute(sql, data) 
    result = cursor.fetchall() # fetchall(): 모든 데이터를 한 번에 가져옴
    db.close()
    return to_pandas(result)

def to_pandas(result):
    return pd.DataFrame(result)

In [3]:
# 기본 예제
# SET @X = '[{"type": "FALL", "value": 1}, {"type": "STEP", "value": 100}]';
# SELECT JSON_SEARCH(@X, 'one', 'FALL');
# SELECT SUBSTRING(JSON_SEARCH(@X, 'one', 'FALL') FROM 2 FOR 4);
# SELECT JSON_EXTRACT(@X, '$[*]');

## 기본 SELECT

In [4]:
# 테이블의 전체 데이터를 select
sql = "SELECT * FROM " + MYSQL_TABLENAME+ ";"
print(sql)
select_all = query(sql, ())
select_all.head()

SELECT * FROM mytable;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,1,b177b428b46311edbb8940b076a1ec6b,2023-02-24 16:52:59,Choi,Choi@mysql.com,010-9876-5432,35,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
1,2,b1778d08b46311edae6040b076a1ec6b,2023-02-24 16:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",,
2,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
3,4,b177b429b46311edbf3540b076a1ec6b,2023-02-24 15:52:59,Choi,Choi@mysql.com,010-9876-5432,29,"[{""search"": ""github""}, {""viewing time"": ""33""}]",,
4,5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,


In [5]:
select_all.info() # 총 144rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   index     145 non-null    int64         
 1   uuid      145 non-null    object        
 2   reg_date  145 non-null    datetime64[ns]
 3   name      145 non-null    object        
 4   email     145 non-null    object        
 5   phone     145 non-null    object        
 6   energy    145 non-null    int64         
 7   label     145 non-null    object        
 8   mark      145 non-null    object        
 9   ext01     145 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 11.5+ KB


## SELECT + 자주쓰는 기본 문법
- DATE, COALESCE

In [6]:
sql = "SELECT DATE('2023-02-24 11:00:00');" # date 생성해서 테이블처럼 만듦
print(sql)
result = query(sql, ())
result.head()

SELECT DATE('2023-02-24 11:00:00');


,DATE('2023-02-24 11:00:00')
0,2023-02-24


In [7]:
sql = "SELECT COALESCE('2023-02-24 11:00:00', 0);" # date 생성해서 테이블처럼 만듦
print(sql)
result = query(sql, ())
result.head()

SELECT COALESCE('2023-02-24 11:00:00', 0);


,"COALESCE('2023-02-24 11:00:00', 0)"
0,2023-02-24 11:00:00


In [8]:
sql = "SELECT COALESCE(null, 10, 5);" # 값이 null이 아니면서 가장 먼저 입력한 10이 나옴. null만 있다면 null만 나옴
print(sql)
result = query(sql, ())
result.head()

SELECT COALESCE(null, 10, 5);


,"COALESCE(null, 10, 5)"
0,10


## SELECT + WHERE + 조건
- 조건이 FALSE이면 SELECT하지 않음

In [9]:
sql = """SELECT "3","4" WHERE TRUE;"""
result = query(sql, ())
print(result)

   3  4
0  3  4


In [10]:
sql = """SELECT "3","4" WHERE FALSE;"""
result = query(sql, ())
print(result)

Empty DataFrame
Columns: []
Index: []


In [11]:
# name 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s;" 
print(sql)
result = query(sql, (['Kim']))
result[['name']].value_counts()

SELECT * FROM mytable where name=%s;


name
Kim     48
dtype: int64

In [12]:
# 시간 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);" 
print(sql)
result = query(sql, (['2023-02-24 00:00:00', '2023-02-25 00:00:00']))
result.sort_values('reg_date')

SELECT * FROM mytable where reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
50,51,b17765f9b46311ed83b740b076a1ec6b,2023-02-24 00:52:59,Kim,Kim@mysql.com,010-1234-5678,35,"[{""search"": ""naver""}, {""viewing time"": ""75""}]",,
49,50,b1778d18b46311edac5340b076a1ec6b,2023-02-24 00:52:59,Lee,Lee@mysql.com,010-4567-8910,83,"[{""search"": ""github""}, {""viewing time"": ""32""}]",,
48,49,b177db27b46311edbe2b40b076a1ec6b,2023-02-24 00:52:59,Choi,Choi@mysql.com,010-9876-5432,16,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
47,48,b17765f8b46311ed805240b076a1ec6b,2023-02-24 01:52:59,Kim,Kim@mysql.com,010-1234-5678,74,"[{""search"": ""youtube""}, {""viewing time"": ""23""}]",,
46,47,b1778d17b46311ed923840b076a1ec6b,2023-02-24 01:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""61""}]",,
45,46,b177db26b46311edaf7c40b076a1ec6b,2023-02-24 01:52:59,Choi,Choi@mysql.com,010-9876-5432,0,"[{""search"": ""naver""}, {""viewing time"": ""63""}]",,
44,45,b17765f7b46311ed88a840b076a1ec6b,2023-02-24 02:52:59,Kim,Kim@mysql.com,010-1234-5678,57,"[{""search"": ""youtube""}, {""viewing time"": ""31""}]",,
43,44,b1778d16b46311ed91c040b076a1ec6b,2023-02-24 02:52:59,Lee,Lee@mysql.com,010-4567-8910,29,"[{""search"": ""github""}, {""viewing time"": ""25""}]",,
42,43,b177db25b46311edb23240b076a1ec6b,2023-02-24 02:52:59,Choi,Choi@mysql.com,010-9876-5432,81,"[{""search"": ""github""}, {""viewing time"": ""58""}]",,
41,42,b17765f6b46311ed899140b076a1ec6b,2023-02-24 03:52:59,Kim,Kim@mysql.com,010-1234-5678,32,"[{""search"": ""github""}, {""viewing time"": ""9""}]",,


In [13]:
# name 지정 + 시간 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s AND reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);" 
print(sql)
result = query(sql, (['Kim', '2023-02-24 12:00:00', '2023-02-25 00:00:00']))
result.sort_values('reg_date')

SELECT * FROM mytable where name=%s AND reg_date BETWEEN TIMESTAMP(%s) AND TIMESTAMP(%s);


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
4,15,b17765edb46311edaaeb40b076a1ec6b,2023-02-24 12:52:59,Kim,Kim@mysql.com,010-1234-5678,88,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",,
3,12,b17765ecb46311ed87ad40b076a1ec6b,2023-02-24 13:52:59,Kim,Kim@mysql.com,010-1234-5678,87,"[{""search"": ""naver""}, {""viewing time"": ""88""}]",,
2,9,b17765ebb46311ed924f40b076a1ec6b,2023-02-24 14:52:59,Kim,Kim@mysql.com,010-1234-5678,21,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
1,6,b1773edab46311ed865640b076a1ec6b,2023-02-24 15:52:59,Kim,Kim@mysql.com,010-1234-5678,67,"[{""search"": ""github""}, {""viewing time"": ""83""}]",,
0,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,


## SELECT + LIMIT
- 조회 개수 제한
- 참고로 String Format을 사용하면 d %s %f 등 지정해주는 것이 맞지만 pymysql에서 변수 타입과 상관없이 전부 %s를 사용해줘야 한다

In [14]:
# name 지정
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s LIMIT %s;" # 변수 타입 상관없이 %s
print(sql)
result = query(sql, (['Kim', 5])) 
result[['name']].value_counts()

SELECT * FROM mytable where name=%s LIMIT %s;


name
Kim     5
dtype: int64

In [15]:
result

,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
1,6,b1773edab46311ed865640b076a1ec6b,2023-02-24 15:52:59,Kim,Kim@mysql.com,010-1234-5678,67,"[{""search"": ""github""}, {""viewing time"": ""83""}]",,
2,9,b17765ebb46311ed924f40b076a1ec6b,2023-02-24 14:52:59,Kim,Kim@mysql.com,010-1234-5678,21,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
3,12,b17765ecb46311ed87ad40b076a1ec6b,2023-02-24 13:52:59,Kim,Kim@mysql.com,010-1234-5678,87,"[{""search"": ""naver""}, {""viewing time"": ""88""}]",,
4,15,b17765edb46311edaaeb40b076a1ec6b,2023-02-24 12:52:59,Kim,Kim@mysql.com,010-1234-5678,88,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",,


## SELECT + ORDER BY + LIMIT N,M
- 오름차순으로 내림/오름 차순으로 정렬 뒤에 N행 부터 M개 행을 가져옴
    - LIMITE N, M: N행에서부터 M개를 가져옴

In [16]:
# LIMIT로 지정한 행을 가져올 수 있음 
sql = "SELECT * FROM " + MYSQL_TABLENAME +" where name=%s ORDER BY reg_date DESC  LIMIT %s, %s;" # 내림차순
print(sql)
result = query(sql, (['Kim', 2, 5])) 
result

SELECT * FROM mytable where name=%s ORDER BY reg_date DESC  LIMIT %s, %s;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,9,b17765ebb46311ed924f40b076a1ec6b,2023-02-24 14:52:59,Kim,Kim@mysql.com,010-1234-5678,21,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
1,12,b17765ecb46311ed87ad40b076a1ec6b,2023-02-24 13:52:59,Kim,Kim@mysql.com,010-1234-5678,87,"[{""search"": ""naver""}, {""viewing time"": ""88""}]",,
2,15,b17765edb46311edaaeb40b076a1ec6b,2023-02-24 12:52:59,Kim,Kim@mysql.com,010-1234-5678,88,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",,
3,18,b17765eeb46311ed901d40b076a1ec6b,2023-02-24 11:52:59,Kim,Kim@mysql.com,010-1234-5678,65,"[{""search"": ""naver""}, {""viewing time"": ""39""}]",,
4,21,b17765efb46311ed8fb840b076a1ec6b,2023-02-24 10:52:59,Kim,Kim@mysql.com,010-1234-5678,87,"[{""search"": ""naver""}, {""viewing time"": ""88""}]",,


## SELECT + COUNT

In [17]:
# row counts
sql = "SELECT COUNT(*) FROM " + MYSQL_TABLENAME +";" # 내림차순
print(sql)
result = query(sql, ()) 
result

SELECT COUNT(*) FROM mytable;


,COUNT(*)
0,145


In [18]:
# row counts + 조건문
sql = "SELECT COUNT(*) FROM " + MYSQL_TABLENAME +" where name=%s;" # 내림차순
print(sql)
result = query(sql, (['Kim'])) 
result

SELECT COUNT(*) FROM mytable where name=%s;


,COUNT(*)
0,48


## SELECT + LIKE
- 특정 문자열이 있는지 검색 LIKE에 붙는 '%'위치에 따라 조건이 다름

In [19]:
# "[{'serach':%": [{'serach':로 시작하는 데이터 검색
sql = "SELECT * FROM " + MYSQL_TABLENAME + " where label LIKE %s;"
print(sql)
result = query(sql, ([ "[{'serach':%" ])) 
result

SELECT * FROM mytable where label LIKE %s;


""


In [20]:
# "%64}]": 64}]로 끝나는 데이터 검색
sql = "SELECT * FROM " + MYSQL_TABLENAME + " where label LIKE %s;"
print(sql)
result = query(sql, ([ "%64}]" ])) 
result

SELECT * FROM mytable where label LIKE %s;


""


In [21]:
# "%[{'serach': 'youtube'}%": [{'serach': 'youtube'}이 포함되는 데이터 검색
sql = "SELECT * FROM " + MYSQL_TABLENAME + " where label LIKE %s;"
print(sql)
result = query(sql, ([ "%[{'serach': 'youtube'}%" ])) 
result

SELECT * FROM mytable where label LIKE %s;


""


## JSON_EXTRACT
- Text가 JSON 방식이면 JSON형태로 조회

In [22]:
sql = "SELECT JSON_EXTRACT(`label`, '$[*]') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT JSON_EXTRACT(`label`, '$[*]') FROM mytable LIMIT 0, 5;


,"JSON_EXTRACT(`label`, '$[*]')"
0,"[{""search"": ""naver""}, {""viewing time"": ""30""}]"
1,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]"
2,"[{""search"": ""github""}, {""viewing time"": ""64""}]"
3,"[{""search"": ""github""}, {""viewing time"": ""33""}]"
4,"[{""search"": ""naver""}, {""viewing time"": ""19""}]"


In [23]:
# 전체에서 .search의 value만 get
sql = "SELECT JSON_EXTRACT(`label`, '$[*].search') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT JSON_EXTRACT(`label`, '$[*].search') FROM mytable LIMIT 0, 5;


,"JSON_EXTRACT(`label`, '$[*].search')"
0,"[""naver""]"
1,"[""youtube""]"
2,"[""github""]"
3,"[""github""]"
4,"[""naver""]"


In [24]:
# JSON_EXTRACT는 ""큰 따옴표를 사용
# 위 array 형태를 $[0]과 같이 정해진 순서 array를 가져올 수 잇음
sql = "SELECT JSON_EXTRACT(`label`, '$[1]') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT JSON_EXTRACT(`label`, '$[1]') FROM mytable LIMIT 0, 5;


,"JSON_EXTRACT(`label`, '$[1]')"
0,"{""viewing time"": ""30""}"
1,"{""viewing time"": ""54""}"
2,"{""viewing time"": ""64""}"
3,"{""viewing time"": ""33""}"
4,"{""viewing time"": ""19""}"


In [25]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT JSON_EXTRACT(JSON_EXTRACT(`label`, '$[1]'), '$.viewing time') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT JSON_EXTRACT(JSON_EXTRACT(`label`, '$[1]'), '$.viewing time') FROM mytable LIMIT 0, 5;


,"JSON_EXTRACT(JSON_EXTRACT(`label`, '$[1]'), '$.viewing time')"
0,"""30"""
1,"""54"""
2,"""64"""
3,"""33"""
4,"""19"""


## JSON_SEARCH
- JSON_SEARCH(json_doc, 'one' or 'all', search str): str을 포함한 하나 혹은 다수의 JSON object 추출

In [26]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') FROM mytable LIMIT 0, 5;


,"JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github')"
0,None
1,None
2,"""$[0].search"""
3,"""$[0].search"""
4,None


In [27]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') as 'key of github' FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') as 'key of github' FROM mytable LIMIT 0, 5;


,key of github
0,None
1,None
2,"""$[0].search"""
3,"""$[0].search"""
4,None


## SUBSTRING
- SUBSTRING(문자열 FROM N for M): N부터 읽고 M개 문자를 읽어옴

In [28]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT SUBSTRING(label FROM 4 for 6) FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT SUBSTRING(label FROM 4 for 6) FROM mytable LIMIT 0, 5;


,SUBSTRING(label FROM 4 for 6)
0,search
1,search
2,search
3,search
4,search


## JSON_SEARCH + SUBSTRING
- JSON_SEARCH + SUBSTRING 두 가지를 사용하여 Object index를 알 수 있음

In [29]:
# array 로부터 json 형식으로 extract 한 번더 수행
sql = "SELECT SUBSTRING(JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') from 4 for 1) FROM " + MYSQL_TABLENAME + " LIMIT 0, 5;"
print(sql)
result = query(sql, ()) 
result

SELECT SUBSTRING(JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') from 4 for 1) FROM mytable LIMIT 0, 5;


,"SUBSTRING(JSON_SEARCH(JSON_EXTRACT(`label`, '$[*]'), 'one', 'github') from 4 for 1)"
0,None
1,None
2,0
3,0
4,None


## GROUP BY
- 그룹화하여 데이터 조회. 단, 그룹화 한 column을 그대로 select에 사용해야함
- GROUP BY에서 자주 사용되는 WHERE 과 HAVING
    - WHERE: 그룹화 하기 전에 조건
    - HAVING: 그룹화 후 조건

In [30]:
select_all.head()

,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,1,b177b428b46311edbb8940b076a1ec6b,2023-02-24 16:52:59,Choi,Choi@mysql.com,010-9876-5432,35,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
1,2,b1778d08b46311edae6040b076a1ec6b,2023-02-24 16:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",,
2,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
3,4,b177b429b46311edbf3540b076a1ec6b,2023-02-24 15:52:59,Choi,Choi@mysql.com,010-9876-5432,29,"[{""search"": ""github""}, {""viewing time"": ""33""}]",,
4,5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,


In [31]:
# energy 90이상인 것들 중에 name과 energy로 그룹화한 것을 조회
sql = "SELECT *, COUNT(energy) as cnt from " + MYSQL_TABLENAME + " where energy > 90 GROUP BY name, energy;"
print(sql)
result = query(sql, ()) 
result

SELECT *, COUNT(energy) as cnt from mytable where energy > 90 GROUP BY name, energy;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01,cnt
0,19,b177b42eb46311eda93340b076a1ec6b,2023-02-24 10:52:59,Choi,Choi@mysql.com,010-9876-5432,91,"[{""search"": ""github""}, {""viewing time"": ""39""}]",,,3
1,34,b177db22b46311eda10640b076a1ec6b,2023-02-24 05:52:59,Choi,Choi@mysql.com,010-9876-5432,93,"[{""search"": ""naver""}, {""viewing time"": ""21""}]",,,1
2,52,b177db28b46311ed9a8640b076a1ec6b,2023-02-23 23:52:59,Choi,Choi@mysql.com,010-9876-5432,94,"[{""search"": ""youtube""}, {""viewing time"": ""63""}]",,,1
3,136,b177db44b46311edb37440b076a1ec6b,2023-02-22 19:52:59,Choi,Choi@mysql.com,010-9876-5432,99,"[{""search"": ""naver""}, {""viewing time"": ""81""}]",,,1
4,114,b1778cfdb46311ed9c2e40b076a1ec6b,2023-02-23 03:52:59,Kim,Kim@mysql.com,010-1234-5678,98,"[{""search"": ""naver""}, {""viewing time"": ""77""}]",,,1
5,36,b17765f4b46311eda47d40b076a1ec6b,2023-02-24 05:52:59,Kim,Kim@mysql.com,010-1234-5678,99,"[{""search"": ""youtube""}, {""viewing time"": ""49""}]",,,1
6,140,b177b426b46311ed84d440b076a1ec6b,2023-02-22 18:52:59,Lee,Lee@mysql.com,010-4567-8910,91,"[{""search"": ""youtube""}, {""viewing time"": ""36""}]",,,1
7,119,b177b41fb46311eda48540b076a1ec6b,2023-02-23 01:52:59,Lee,Lee@mysql.com,010-4567-8910,97,"[{""search"": ""youtube""}, {""viewing time"": ""34""}]",,,1
8,159,8f75f6928f034d49959e58242a7fc7ad,2023-02-25 19:20:12,test,test@mytable.com,123-4567-8910,100,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,,1


In [32]:
sql = "SELECT * FROM " + MYSQL_TABLENAME + """ where JSON_EXTRACT(label, '$[1]."viewing time"')>=90;"""
print(sql)
result = query(sql, ()) 
result

SELECT * FROM mytable where JSON_EXTRACT(label, '$[1]."viewing time"')>=90;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,7,b177b42ab46311edb3e840b076a1ec6b,2023-02-24 14:52:59,Choi,Choi@mysql.com,010-9876-5432,18,"[{""search"": ""github""}, {""viewing time"": ""93""}]",,
1,17,b1778d0db46311ed814e40b076a1ec6b,2023-02-24 11:52:59,Lee,Lee@mysql.com,010-4567-8910,3,"[{""search"": ""naver""}, {""viewing time"": ""98""}]",,
2,37,b177db23b46311edab2140b076a1ec6b,2023-02-24 04:52:59,Choi,Choi@mysql.com,010-9876-5432,73,"[{""search"": ""github""}, {""viewing time"": ""96""}]",,
3,65,b1778d1db46311ed88df40b076a1ec6b,2023-02-23 19:52:59,Lee,Lee@mysql.com,010-4567-8910,11,"[{""search"": ""naver""}, {""viewing time"": ""93""}]",,
4,75,b1776601b46311ed930340b076a1ec6b,2023-02-23 16:52:59,Kim,Kim@mysql.com,010-1234-5678,46,"[{""search"": ""naver""}, {""viewing time"": ""91""}]",,
5,78,b1776602b46311ed9a6d40b076a1ec6b,2023-02-23 15:52:59,Kim,Kim@mysql.com,010-1234-5678,0,"[{""search"": ""naver""}, {""viewing time"": ""99""}]",,
6,93,b1776607b46311edb93d40b076a1ec6b,2023-02-23 10:52:59,Kim,Kim@mysql.com,010-1234-5678,78,"[{""search"": ""youtube""}, {""viewing time"": ""99""}]",,
7,102,b177660ab46311eda95940b076a1ec6b,2023-02-23 07:52:59,Kim,Kim@mysql.com,010-1234-5678,41,"[{""search"": ""youtube""}, {""viewing time"": ""95""}]",,
8,104,b177b41ab46311ed913b40b076a1ec6b,2023-02-23 06:52:59,Lee,Lee@mysql.com,010-4567-8910,41,"[{""search"": ""naver""}, {""viewing time"": ""90""}]",,
9,112,b177db3cb46311ed9eeb40b076a1ec6b,2023-02-23 03:52:59,Choi,Choi@mysql.com,010-9876-5432,53,"[{""search"": ""youtube""}, {""viewing time"": ""94""}]",,


In [33]:
sql = "SELECT * from " + MYSQL_TABLENAME + " where JSON_EXTRACT(label, '$[1].viewing time')>=90;"
print(sql)
result = query(sql, ()) 
result

SELECT * from mytable where JSON_EXTRACT(label, '$[1].viewing time')>=90;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,7,b177b42ab46311edb3e840b076a1ec6b,2023-02-24 14:52:59,Choi,Choi@mysql.com,010-9876-5432,18,"[{""search"": ""github""}, {""viewing time"": ""93""}]",,
1,17,b1778d0db46311ed814e40b076a1ec6b,2023-02-24 11:52:59,Lee,Lee@mysql.com,010-4567-8910,3,"[{""search"": ""naver""}, {""viewing time"": ""98""}]",,
2,37,b177db23b46311edab2140b076a1ec6b,2023-02-24 04:52:59,Choi,Choi@mysql.com,010-9876-5432,73,"[{""search"": ""github""}, {""viewing time"": ""96""}]",,
3,65,b1778d1db46311ed88df40b076a1ec6b,2023-02-23 19:52:59,Lee,Lee@mysql.com,010-4567-8910,11,"[{""search"": ""naver""}, {""viewing time"": ""93""}]",,
4,75,b1776601b46311ed930340b076a1ec6b,2023-02-23 16:52:59,Kim,Kim@mysql.com,010-1234-5678,46,"[{""search"": ""naver""}, {""viewing time"": ""91""}]",,
5,78,b1776602b46311ed9a6d40b076a1ec6b,2023-02-23 15:52:59,Kim,Kim@mysql.com,010-1234-5678,0,"[{""search"": ""naver""}, {""viewing time"": ""99""}]",,
6,93,b1776607b46311edb93d40b076a1ec6b,2023-02-23 10:52:59,Kim,Kim@mysql.com,010-1234-5678,78,"[{""search"": ""youtube""}, {""viewing time"": ""99""}]",,
7,102,b177660ab46311eda95940b076a1ec6b,2023-02-23 07:52:59,Kim,Kim@mysql.com,010-1234-5678,41,"[{""search"": ""youtube""}, {""viewing time"": ""95""}]",,
8,104,b177b41ab46311ed913b40b076a1ec6b,2023-02-23 06:52:59,Lee,Lee@mysql.com,010-4567-8910,41,"[{""search"": ""naver""}, {""viewing time"": ""90""}]",,
9,112,b177db3cb46311ed9eeb40b076a1ec6b,2023-02-23 03:52:59,Choi,Choi@mysql.com,010-9876-5432,53,"[{""search"": ""youtube""}, {""viewing time"": ""94""}]",,


In [34]:
# viewing time이 90 이상인 사람 중에 search 그룹화 
sql = "SELECT *, COUNT(JSON_EXTRACT(label, '$[0].search')) as cnt from " + MYSQL_TABLENAME + " where JSON_EXTRACT(label, '$[1].viewing time')>=90 GROUP BY JSON_EXTRACT(label, '$[0].search');"
print(sql)
result = query(sql, ()) 
result

SELECT *, COUNT(JSON_EXTRACT(label, '$[0].search')) as cnt from mytable where JSON_EXTRACT(label, '$[1].viewing time')>=90 GROUP BY JSON_EXTRACT(label, '$[0].search');


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01,cnt
0,7,b177b42ab46311edb3e840b076a1ec6b,2023-02-24 14:52:59,Choi,Choi@mysql.com,010-9876-5432,18,"[{""search"": ""github""}, {""viewing time"": ""93""}]",,,2
1,17,b1778d0db46311ed814e40b076a1ec6b,2023-02-24 11:52:59,Lee,Lee@mysql.com,010-4567-8910,3,"[{""search"": ""naver""}, {""viewing time"": ""98""}]",,,7
2,93,b1776607b46311edb93d40b076a1ec6b,2023-02-23 10:52:59,Kim,Kim@mysql.com,010-1234-5678,78,"[{""search"": ""youtube""}, {""viewing time"": ""99""}]",,,3


## INNER JOIN
- SELECT <열 목록> FROM <기준 테이블> INNER JOIN <참조 테이블> ON <조인 조건> [WHERE 검색 조건]
- 두 개 이상의 테이블을 묶어서 하나의 결과 집합을 만들어 내는 것. 그냥 JOIN 으로 써도 INNER JOIN 으로 인식함
- JOIN은 두 테이블의 교집합. 동시에 같은 데이터 필드가 있으면 합쳐서 포함
    - 동일 조건 비교면 <b><u>데이터 필드로 비교해야함</u></b>. 결국엔 데이터 필드 기준으로 비교

아래서는 하나의 테이블을 활용하여 서로 다른 테이블 처럼 활용

In [35]:
# 그룹화한 테이블1
sql = "SELECT NAME AS n, JSON_EXTRACT(label, '$[0].search') AS site FROM " + MYSQL_TABLENAME + " GROUP BY NAME, JSON_EXTRACT(label, '$[0].search');"
print(sql)
result = query(sql, ()) 
result

SELECT NAME AS n, JSON_EXTRACT(label, '$[0].search') AS site FROM mytable GROUP BY NAME, JSON_EXTRACT(label, '$[0].search');


,n,site
0,Choi,"""github"""
1,Choi,"""naver"""
2,Choi,"""youtube"""
3,Kim,"""github"""
4,Kim,"""naver"""
5,Kim,"""youtube"""
6,Lee,"""github"""
7,Lee,"""naver"""
8,Lee,"""youtube"""
9,test,"""naver"""


In [36]:
# 그룹화한 테이블1과 기존 테이블
# 같이 비교할 수 있는 필드가 있어야함. 필드끼리 비교
sql = "SELECT * FROM (SELECT NAME AS n, label AS grouplabel FROM " + MYSQL_TABLENAME + " GROUP BY NAME, JSON_EXTRACT(label, '$[0].search')) as t1\
 INNER JOIN " + MYSQL_TABLENAME + " ON label=t1.grouplabel;"
print(sql)
result = query(sql, ()) 
result

SELECT * FROM (SELECT NAME AS n, label AS grouplabel FROM mytable GROUP BY NAME, JSON_EXTRACT(label, '$[0].search')) as t1 INNER JOIN mytable ON label=t1.grouplabel;


,n,grouplabel,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,Choi,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",1,b177b428b46311edbb8940b076a1ec6b,2023-02-24 16:52:59,Choi,Choi@mysql.com,010-9876-5432,35,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
1,Lee,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",2,b1778d08b46311edae6040b076a1ec6b,2023-02-24 16:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",,
2,Kim,"[{""search"": ""github""}, {""viewing time"": ""64""}]",3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
3,Choi,"[{""search"": ""github""}, {""viewing time"": ""33""}]",4,b177b429b46311edbf3540b076a1ec6b,2023-02-24 15:52:59,Choi,Choi@mysql.com,010-9876-5432,29,"[{""search"": ""github""}, {""viewing time"": ""33""}]",,
4,Lee,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,
5,test,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,
6,Kim,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",9,b17765ebb46311ed924f40b076a1ec6b,2023-02-24 14:52:59,Kim,Kim@mysql.com,010-1234-5678,21,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
7,Kim,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",15,b17765edb46311edaaeb40b076a1ec6b,2023-02-24 12:52:59,Kim,Kim@mysql.com,010-1234-5678,88,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",,
8,Choi,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",22,b177b42fb46311edb69340b076a1ec6b,2023-02-24 09:52:59,Choi,Choi@mysql.com,010-9876-5432,39,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
9,Choi,"[{""search"": ""youtube""}, {""viewing time"": ""43""}]",28,b177b431b46311eda5c040b076a1ec6b,2023-02-24 07:52:59,Choi,Choi@mysql.com,010-9876-5432,18,"[{""search"": ""youtube""}, {""viewing time"": ""43""}]",,


In [37]:
# 그룹화한 테이블1과 기존 테이블 + WHERE 조건 까지
# 같이 비교할 수 있는 필드가 있어야함. 필드끼리 비교
sql = "SELECT * FROM (SELECT NAME AS n, label AS grouplabel FROM " + MYSQL_TABLENAME + " WHERE NAME='KIM' GROUP BY NAME, JSON_EXTRACT(label, '$[0].search')) as t1\
 INNER JOIN " + MYSQL_TABLENAME + " ON label=t1.grouplabel;"
print(sql)
result = query(sql, ()) 
result

SELECT * FROM (SELECT NAME AS n, label AS grouplabel FROM mytable WHERE NAME='KIM' GROUP BY NAME, JSON_EXTRACT(label, '$[0].search')) as t1 INNER JOIN mytable ON label=t1.grouplabel;


,n,grouplabel,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,Kim,"[{""search"": ""github""}, {""viewing time"": ""64""}]",3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
1,Kim,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",9,b17765ebb46311ed924f40b076a1ec6b,2023-02-24 14:52:59,Kim,Kim@mysql.com,010-1234-5678,21,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
2,Kim,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",15,b17765edb46311edaaeb40b076a1ec6b,2023-02-24 12:52:59,Kim,Kim@mysql.com,010-1234-5678,88,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",,
3,Kim,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",49,b177db27b46311edbe2b40b076a1ec6b,2023-02-24 00:52:59,Choi,Choi@mysql.com,010-9876-5432,16,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
